# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770848812875                   -0.52    9.0    180ms
  2   -2.772147460715       -2.89       -1.32    1.0    118ms
  3   -2.772170194783       -4.64       -2.43    1.0    109ms
  4   -2.772170648329       -6.34       -3.13    1.0    113ms
  5   -2.772170722716       -7.13       -4.29    2.0    129ms
  6   -2.772170722849       -9.88       -4.48    1.0    119ms
  7   -2.772170723012       -9.79       -5.40    1.0    120ms
  8   -2.772170723014      -11.67       -5.56    2.0    135ms
  9   -2.772170723015      -12.04       -6.55    1.0    539ms
 10   -2.772170723015      -14.15       -6.61    2.0    130ms
 11   -2.772170723015      -14.18       -7.14    1.0    126ms
 12   -2.772170723015      -14.75       -7.96    1.0    135ms
 13   -2.772170723015   +  -14.03       -8.51    2.0    133ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735582112779569

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770793143969                   -0.52    9.0    178ms
  2   -2.772061188153       -2.90       -1.32    1.0    109ms
  3   -2.772082991222       -4.66       -2.43    1.0    111ms
  4   -2.772083338990       -6.46       -3.13    1.0    115ms
  5   -2.772083417699       -7.10       -4.42    2.0    135ms
  6   -2.772083417757      -10.23       -4.64    1.0    116ms
  7   -2.772083417809      -10.29       -5.72    1.0    539ms
  8   -2.772083417811      -11.71       -6.34    2.0    128ms
  9   -2.772083417811      -13.83       -7.75    1.0    113ms
 10   -2.772083417811      -13.80       -8.22    2.0    134ms

Polarizability via ForwardDiff:       1.7725349863786195
Polarizability via finite difference: 1.7735582112779569
